<b>Exercise 8: Realistic morphology.</b><br><br>
We import the neuron morphology from a 3-D reconstructed "Hay-model" neuron (https://senselab.med.yale.edu/modeldb/showModel.cshtml?model=139653). The ion channels and their distribution are determined as in the model implementation in the above link - this is described in L5PCbiophys.hoc.

In [1]:
#Import the python libraries
import matplotlib.pyplot as plt
plt.ion()
import matplotlib.cm as cm
from neuron import h
import numpy
import time
from ipywidgets import widgets, fixed, Layout
import pickle
unpicklefile = open('coords.sav', 'rb')
unpickledlist = pickle.load(unpicklefile)
unpicklefile.close()
xs = numpy.array(unpickledlist[0]); ys = numpy.array(unpickledlist[1]); sections = unpickledlist[2]


In [2]:
%%html
<style type='text/css'>
.widget-inline-hbox .widget-label {
      max-width: 250px;
      min-width: 250px;
}
</style>

In [3]:
### Run NEURON with Python interface
def runneuron(isadaptive=True,amp=0.12,dur=10,stimcomp=0,reccomp=0):
    my_start_time = time.time()
        
    h.load_file("stdlib.hoc")                        #A default NEURON library
    h.load_file("stdrun.hoc")                        #A default NEURON library
    h.load_file("import3d.hoc")                      #A default NEURON library needed for importing 3D morphologies

    h('objref cvode, L5PC')                          #Define an object cvode
    h('cvode = new CVode()')                         #Make cvode a time step integrator object
    h('cvode.active('+str(isadaptive*1)+')')         #Set the variable time step integration on/off

    h.load_file("L5PCbiophys.hoc")                   #Load the biophysical properties (ion channel conductances etc.)                                                                                                                      
    h.load_file("L5PCtemplate.hoc")                  #Load a template for L5PCs                                                                                                                                                            
    h("L5PC = new L5PCtemplate(\"cell1.asc\")")      #Make an instance of L5PCtemplate. Give as an argument the name                                                                                                                       
                                                     #of the morphology file, which will be loaded                                                                                                                                         
    h('access L5PC.soma')                            #Make soma the currently accessed section                                                                                                                                             

    h.dt = 0.025                                     #Set the time step to 0.025 ms                                                                                                                                                        
    h.tstop = 200                                    #Continue the simulation until 200 ms          
    
    h('objref stim, Vrec')                           #Declare objects stim and Vrec

    #Make stim an IClamp object, stimulating at a given location of a given compartment
    h('L5PC.'+sections[stimcomp]+' stim = new IClamp(0.5)')            
    
    h.stim.delay = 100                               #Stimulation starts at 100 ms
    h.stim.dur = dur                                 #and lasts 10 ms
    h.stim.amp = amp                                 #and has a given amplitude (default 0.075 nA)

    h('objref Vrec, trec')
    h('Vrec = new Vector()')
    h('trec = new Vector()')
    h('L5PC.soma cvode.record(&v(0.5),Vrec,trec)')

    Vrecs = []                                       #Save segment-wise membrane potential values in Vrecs
    for i in range(0,h.L5PC.soma[0].nseg):           #Iterate across soma
        Vrecs.append(h.Vector())                     #Append a vector and record to the added vector
        Vrecs[-1].record(h.L5PC.soma[0]((i+0.5)/h.L5PC.soma[0].nseg)._ref_v, sec=h.L5PC.soma[0])       
    for j in range(0,84):                            #Iterate across basal tree
        for i in range(0,h.L5PC.dend[j].nseg):       #Iterate across a basal section
            Vrecs.append(h.Vector())                 #Append a vector and record to the added vector
            Vrecs[-1].record(h.L5PC.dend[j]((i+0.5)/h.L5PC.dend[j].nseg)._ref_v, sec=h.L5PC.dend[j])       
    for j in range(0,109):                           #Iterate across apical tree
        for i in range(0,h.L5PC.apic[j].nseg):       #Iterate across a apical section
            Vrecs.append(h.Vector())                 #Append a vector and record to the added vector
            Vrecs[-1].record(h.L5PC.apic[j]((i+0.5)/h.L5PC.apic[j].nseg)._ref_v, sec=h.L5PC.apic[j])       
    
    h.init()                                         #Initialize state variables
    h.run()                                          #Run the simulations

    itime = next(i for i,x in enumerate(numpy.array(h.trec)) if x >= 100)
    maxvs = numpy.array([max(numpy.array(x)[itime:]) for x in Vrecs])
    vdiffs = [-85+5*x for x in range(0,26)]
    colors = cm.rainbow(numpy.linspace(0, 1, 26))
        
    
    plt.close("all")
    f, axs = plt.subplots(1, 2, sharey=False)
    axs[0].plot(h.trec,Vrecs[reccomp])
    axs[0].set_ylabel('$V_m(t)$ (mV)')
    for icolor in range(0,25):
        inds = numpy.logical_and(maxvs > vdiffs[icolor], maxvs <= vdiffs[icolor+1])
        axs[1].plot(xs[inds],ys[inds],'b.',color=colors[icolor])
        axs[1].plot(240,400+20*icolor,'bs',color=colors[icolor],mew=2)
        if icolor % 4 == 0:
            axs[1].text(100,400+20*icolor,str(vdiffs[icolor])+' mV')
    axs[1].plot(240,400+20*28,'bs',color=colors[25],mew=2)
    axs[1].text(100,400+20*28,'>'+str(vdiffs[25])+' mV')
    inds = maxvs > vdiffs[25]
    axs[1].plot(xs[inds],ys[inds],'b.',color=colors[25])
        

    axs[1].plot(xs[reccomp],ys[reccomp],'ko',markersize=6,mew=4)
    axs[1].plot(xs[stimcomp],ys[stimcomp],'gx',markersize=12,mew=4)
    
    plt.suptitle("Simulation run in "+str(time.time() - my_start_time)+" seconds.")
    plt.show()


In [4]:
slider = widgets.interact(runneuron, isadaptive=fixed(True), amp=widgets.FloatSlider(min=0.0,max=8.0,step=0.01,value=0.12,description='Amplitude of stimulus: amp',layout=Layout(width='50%'), continuous_update = False),
                          dur=widgets.FloatSlider(min=0.0,max=100.0,step=1.0,value=10,description='Duration of stimulus: dur',layout=Layout(width='50%'), continuous_update = False),
                          stimcomp=widgets.IntSlider(min=0,max=639,step=1,value=0,description='Stimulated compartment index: stimcomp',layout=Layout(width='50%'), continuous_update = False), 
                          reccomp=widgets.IntSlider(min=0,max=639,step=1,value=0,description='Recorded compartment index: stimcomp',layout=Layout(width='50%'), continuous_update = False))


interactive(children=(FloatSlider(value=0.12, continuous_update=False, description='Amplitude of stimulus: amp…

In this exercise, a reconstructed layer V pyramidal cell (L5PC) morphology was loaded and used in the simulation. The left-hand panel shows the membrane potential in the middle of a selected compartment. The right-hand panel illustrates the neuron morphology. The green 'x' points at the stimulated compartment, and the black dot points at the recorded compartment (whose membrane potential is shown in left-hand panel). The colors of the other points in the morphology show the maximal membrane potential in the underlying compartment during the simulation.

#Question: Stimulate the neuron at the distal apical dendrite (around compartments 398-413). Can you make the neuron fire an action potential? Is this realistic?